# Download and Unpack

In [5]:
import zipfile
import os
import requests

In [1]:
# Replace 'your_api_key' with your actual API key and 'file_id' with the file ID you want to download
url = "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/47317/5799376/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1684949245&Signature=G3Av7r1wXj6qoN9RkZmsmspP4tI0P4vkjdh1YDn3Q3VQUNvTqDCrEHhYtA64WidUi9xbO4%2FdrvkdW21zezgaFsQVNiwv56f5LjAEvScND4i3NlnCpdNE48UQuJWPWyxbnCP1mgF8mA71qc9z3G4xRef9WMVAKjXKSJnLxi2ioeFFD63ZJU%2BIWFMZVILYGMKg1GkA2kG%2FaZKTdXzQhN9nL3Dc%2BQPl31Qd2Gj9v1ou044W0hMBhqFufjyu4NYr48ycBAe%2BWWi64i9CdRNlmKMJ4%2BwVfuckR5AtQOxmPDQmySSHDhLXvoxLxDSZncdbAu%2BJm8PifSct4I8x45rTSkSBPQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dvesuvius-challenge-ink-detection.zip"
response = requests.get(url)

# Save the downloaded file to a local file
with open("downloaded_file.zip", "wb") as f:
    f.write(response.content)

In [2]:
# Unpack a .zip file to a folder
with zipfile.ZipFile('downloaded_file.zip', 'r') as zip_ref:
    zip_ref.extractall('./raw')


# Make split folds and Convert to uint8 png

In [20]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import os, shutil

import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

from PIL import Image
import cv2

In [5]:
os.makedirs('./folds', exist_ok=True)

def load_image(filename):
    return cv2.imread(filename, 0)

## 13

In [11]:
parent_dir = "./raw/train"
train_dir_list = [d for d in glob(f"{parent_dir}/*") if '/2' not in d]

for train_dir in train_dir_list:
    
    print(f'Working on {train_dir}')
    folder_prefix = train_dir.split('raw/')[-1].replace('/', '_')
    os.makedirs(f'./folds/{folder_prefix}', exist_ok=True)
    # Get a list of image file names in the directory
    img_fn_list = sorted(glob(f'{train_dir}/surface_volume/*'))
    
    # Load the images and save as png
    for fn in tqdm(img_fn_list):
        image = load_image(fn)
        Image.fromarray(image).save(f"./folds/{folder_prefix}/surface_{fn.split('/')[-1].split('.')[0]}.png")
    
    shutil.copyfile(f"{train_dir}/mask.png", f'./folds/{folder_prefix}/frag_mask.png')
    shutil.copyfile(f"{train_dir}/inklabels.png", f'./folds/{folder_prefix}/ink_mask.png')

Working on ./raw/train/3


100%|███████████████████████████████████████████| 65/65 [01:54<00:00,  1.76s/it]


Working on ./raw/train/1


100%|███████████████████████████████████████████| 65/65 [02:06<00:00,  1.95s/it]


## 2

In [12]:
train_dir = "./raw/train/2"
folder_prefix = train_dir.split('raw/')[-1].replace('/', '_')
# Get a list of image file names in the directory
img_fn_list = sorted(glob(f'{train_dir}/surface_volume/*'))

In [14]:
split_h = [6100, 10600,] 
frag_mask = load_image(f"{train_dir}/mask.png").astype(bool).astype(np.uint8)
ink_mask = load_image(f"{train_dir}/inklabels.png").astype(bool).astype(np.uint8)

frag_temp_mask_list = np.split(frag_mask, split_h, axis=0)
ink_temp_mask_list = np.split(ink_mask, split_h, axis=0)

# cut empty margins
frag_mask_list, ink_mask_list, pos_list = [],[],[]
for fold_idx in range(3):
    
    os.makedirs(f'./folds/{folder_prefix}{chr(fold_idx+97)}', exist_ok=True)
    rows, cols = np.where(frag_temp_mask_list[fold_idx])
    pos_list.append((rows.min(),rows.max()+1, cols.min(),cols.max()+1))
    # extract the non-empty parts
    frag_mask_list.append(frag_temp_mask_list[fold_idx][rows.min():rows.max()+1, cols.min():cols.max()+1])
    Image.fromarray(frag_mask_list[fold_idx]).save(f'./folds/{folder_prefix}{chr(fold_idx+97)}/frag_mask.png')
    ink_mask_list.append(ink_temp_mask_list[fold_idx][rows.min():rows.max()+1, cols.min():cols.max()+1])
    Image.fromarray(ink_mask_list[fold_idx]).save(f'./folds/{folder_prefix}{chr(fold_idx+97)}/ink_mask.png')
    
del frag_temp_mask_list, ink_temp_mask_list

In [15]:
# load images and make splits
for file_idx,fn in tqdm(enumerate(img_fn_list), total=len(img_fn_list)):
    
    slice_image = load_image(fn)
    slice_image_list = np.split(slice_image, split_h, axis=0)
    
    for fold_idx in range(3):
        r1,r2,c1,c2 = pos_list[fold_idx]
        Image.fromarray(slice_image_list[fold_idx][r1:r2, c1:c2]).save(
            f'./folds/{folder_prefix}{chr(fold_idx+97)}/surface_{file_idx:02d}.png'
        )

100%|███████████████████████████████████████████| 65/65 [07:56<00:00,  7.34s/it]
